In [29]:
#Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
from collections import OrderedDict

## Finding player ids

In [137]:
india = requests.get('https://www.cricbuzz.com/cricket-team/india/2/players')
#Fetch webpage
soup_india = BeautifulSoup(india.content,"html.parser")

In [108]:
print(soup_india.prettify())

<!DOCTYPE html>
<html itemscope="" itemtype="http://schema.org/WebPage" lang="en">
 <head>
  <meta charset="utf-8"/>
  <script>
   var is_mobile = /symbian|tizen|midp|uc(web|browser)|MSIE (5.0|6.0|7.0|8.0)|tablet/i.test(navigator.userAgent);	if(is_mobile && window.location.hostname != "www1.cricbuzz.com") window.location.hostname = "m.cricbuzz.com";
  </script>
  <style>
   body{background:#E3E6E3; font-family: helvetica,"Segoe UI",Arial,sans-serif;color:#222;font-size:14px; line-height: 1.5; margin:0;}	body, .cb-comm-pg, .cb-hm-mid {min-height:1000px}	.container{width:980px;margin:0 auto;}	.page{max-width: 980px;margin: 0 auto;position: relative;}	.cb-col-8 {width:8%;}	.cb-col-10 {width:10%;}	.cb-col-14 {width:14%;}	.cb-col-16 {width:16%;}	.cb-col-20 {width:20%;}	.cb-col-25 {width:25%;}	.cb-col-27 {width:27%;}	.cb-col-33 {width:33%;}	.cb-col-40 {width:40%;}	.cb-col-46 {width:46%;}	.cb-col-47 {width:47%;}	.cb-col-50 {width:50%;}	.cb-col-60 {width:60%;}	.cb-col-66 {width:66%;}	.cb-col-6

In [138]:
id_india = soup_india.find("a", {})

In [139]:
for link in soup_india.findAll('a'):
    print(link.get('href'))

https://plus.google.com/104502282508811467249
Javascript:void(0)
Javascript:void(0)
/
/cricket-match/live-scores
/cricket-schedule/upcoming-series/international
/cricket-scorecard-archives
/cricket-news
/cricket-news
/cricket-news/latest-news
/cricket-news/info/
/cricket-news/editorial/spotlight
/cricket-news/editorial/editorial-list
/cricket-news/editorial/specials
/cricket-news/editorial/stats-analysis
/cricket-news/editorial/interviews
/cricket-news/editorial/live-blogs
/cricket-news/experts/harsha-bhogle/170
/cricket-schedule/series
/cricket-series/2886/west-indies-tour-of-england-2020
/cricket-series/2889/ireland-tour-of-england-2020
/cricket-series/2888/pakistan-tour-of-england-2020
/cricket-series/3250/bob-willis-trophy-2020
/cricket-series/3130/indian-premier-league-2020
/cricket-series/3243/caribbean-premier-league-2020
/cricket-series/2798/icc-mens-t20-world-cup-2020-postponed
/cricket-schedule/series
/cricket-team
/cricket-team/india/2
/cricket-team/ireland/27
/cricket-team/

Manually Copied the player ids from above hrefs into `player_id.csv` file 

In [48]:
#Putting the player ids into a pandas dataframe
player_id = pd.read_csv("player_urls2.csv", header=None)
player_id.columns = ['url']
player_id.reset_index(drop=True, inplace=True)

## Finding attributes

In [49]:
#Fetch data of Player's ID
ids = player_id["url"]

#Prive a base url and an empty list
base_url ="https://www.cricbuzz.com"
player_list = []

#Iterate to scrap data of players from cricbuzz.com
for pages in ids:
    #Using OrderedDict instead of Dict (because we want the same order as we fetch the data and not the sorted one)
    d=OrderedDict()
    #Fetching URLs one by one
    print(base_url+str(pages))
    request = requests.get(base_url+str(pages))
    #Data processing
    content = request.content
    soup = BeautifulSoup(content,"html.parser")

    ## Personal info and icc ranking
    d['player_name'] = soup.find("h1",{"class":"cb-font-40"}).text.replace('/n','')
    d['player_dob'] = soup.find("div",{"class":"cb-col cb-col-60 cb-lst-itm-sm"}).text.replace('/n','')
    d['player_height'] = soup.find("div", {"class":"cb-col cb-col-60"}).text.replace('/n','')
    d['player_role'] = soup.find_all("div", {"class":"cb-col cb-col-60 cb-lst-itm-sm"})[2].text.replace('/n','')
    d['player_batstyle'] = soup.find_all("div", {"class":"cb-col cb-col-60 cb-lst-itm-sm"})[3].text.replace('/n','')
    d['player_bowlingstyle'] = soup.find_all("div", {"class":"cb-col cb-col-60 cb-lst-itm-sm"})[4].text.replace('/n','')
    d['player_bat_test_rank'] = soup.find_all("div", {"class":"cb-col cb-col-25 cb-plyr-rank text-right"})[0].text.replace('/n','')
    d['player_bat_odi_rank'] = soup.find_all("div", {"class":"cb-col cb-col-25 cb-plyr-rank text-right"})[1].text.replace('/n','')
    d['player_bat_t20_rank'] = soup.find_all("div", {"class":"cb-col cb-col-25 cb-plyr-rank text-right"})[2].text.replace('/n','')
    d['player_bowl_test_rank'] = soup.find_all("div", {"class":"cb-col cb-col-25 cb-plyr-rank text-right"})[3].text.replace('/n','')
    d['player_bowl_odi_rank'] = soup.find_all("div", {"class":"cb-col cb-col-25 cb-plyr-rank text-right"})[4].text.replace('/n','')
    d['player_bowl_t20_rank'] = soup.find_all("div", {"class":"cb-col cb-col-25 cb-plyr-rank text-right"})[5].text.replace('/n','')
    d['player_career'] = soup.find_all("div", {"class":"cb-col cb-col-60 cb-lst-itm-sm"})[5].text.replace('/n','')

    ## Batting summary

    #Test
    d['player_test_matches'] = soup.find_all("td", {"class":"text-right"})[0].text.replace('/n','')
    d['player_test_innings'] = soup.find_all("td", {"class":"text-right"})[1].text.replace('/n','')
    d['player_test_notout'] = soup.find_all("td", {"class":"text-right"})[2].text.replace('/n','')
    d['player_test_runs'] = soup.find_all("td", {"class":"text-right"})[3].text.replace('/n','')
    d['player_test_highestscore'] = soup.find_all("td", {"class":"text-right"})[4].text.replace('/n','')
    d['player_test_batavg'] = soup.find_all("td", {"class":"text-right"})[5].text.replace('/n','')
    d['player_test_ballsfaced'] = soup.find_all("td", {"class":"text-right"})[6].text.replace('/n','')
    d['player_test_strikerate'] = soup.find_all("td", {"class":"text-right"})[7].text.replace('/n','')
    d['player_test_100'] = soup.find_all("td", {"class":"text-right"})[8].text.replace('/n','')
    d['player_test_200'] = soup.find_all("td", {"class":"text-right"})[9].text.replace('/n','')
    d['player_test_50'] = soup.find_all("td", {"class":"text-right"})[10].text.replace('/n','')
    d['player_test_4s'] = soup.find_all("td", {"class":"text-right"})[11].text.replace('/n','')
    d['player_test_6s'] = soup.find_all("td", {"class":"text-right"})[12].text.replace('/n','')

    #ODI
    d['player_ODI_matches'] = soup.find_all("td", {"class":"text-right"})[13].text.replace('/n','')
    d['player_ODI_innings'] = soup.find_all("td", {"class":"text-right"})[14].text.replace('/n','')
    d['player_ODI_notout'] = soup.find_all("td", {"class":"text-right"})[15].text.replace('/n','')
    d['player_ODI_runs'] = soup.find_all("td", {"class":"text-right"})[16].text.replace('/n','')
    d['player_ODI_highestscore'] = soup.find_all("td", {"class":"text-right"})[17].text.replace('/n','')
    d['player_ODI_batavg'] = soup.find_all("td", {"class":"text-right"})[18].text.replace('/n','')
    d['player_ODI_ballsfaced'] = soup.find_all("td", {"class":"text-right"})[19].text.replace('/n','')
    d['player_ODI_strikerate'] = soup.find_all("td", {"class":"text-right"})[20].text.replace('/n','')
    d['player_ODI_100'] = soup.find_all("td", {"class":"text-right"})[21].text.replace('/n','')
    d['player_ODI_200'] = soup.find_all("td", {"class":"text-right"})[22].text.replace('/n','')
    d['player_ODI_50'] = soup.find_all("td", {"class":"text-right"})[23].text.replace('/n','')
    d['player_ODI_4s'] = soup.find_all("td", {"class":"text-right"})[24].text.replace('/n','')
    d['player_ODI_6s'] = soup.find_all("td", {"class":"text-right"})[25].text.replace('/n','')

    #t20
    d['player_T20_matches'] = soup.find_all("td", {"class":"text-right"})[26].text.replace('/n','')
    d['player_T20_innings'] = soup.find_all("td", {"class":"text-right"})[27].text.replace('/n','')
    d['player_T20_notout'] = soup.find_all("td", {"class":"text-right"})[28].text.replace('/n','')
    d['player_T20_runs'] = soup.find_all("td", {"class":"text-right"})[29].text.replace('/n','')
    d['player_T20_highestscore'] = soup.find_all("td", {"class":"text-right"})[30].text.replace('/n','')
    d['player_T20_batavg'] = soup.find_all("td", {"class":"text-right"})[31].text.replace('/n','')
    d['player_T20_ballsfaced'] = soup.find_all("td", {"class":"text-right"})[32].text.replace('/n','')
    d['player_T20_strikerate'] = soup.find_all("td", {"class":"text-right"})[33].text.replace('/n','')
    d['player_T20_100'] = soup.find_all("td", {"class":"text-right"})[34].text.replace('/n','')
    d['player_T20_200'] = soup.find_all("td", {"class":"text-right"})[35].text.replace('/n','')
    d['player_T20_50'] = soup.find_all("td", {"class":"text-right"})[36].text.replace('/n','')
    d['player_T20_4s'] = soup.find_all("td", {"class":"text-right"})[37].text.replace('/n','')
    d['player_T20_6s'] = soup.find_all("td", {"class":"text-right"})[38].text.replace('/n','')

    #IPL
    d['player_IPL_matches'] = soup.find_all("td", {"class":"text-right"})[39].text.replace('/n','')
    d['player_IPL_innings'] = soup.find_all("td", {"class":"text-right"})[40].text.replace('/n','')
    d['player_IPL_notout'] = soup.find_all("td", {"class":"text-right"})[41].text.replace('/n','')
    d['player_IPL_runs'] = soup.find_all("td", {"class":"text-right"})[42].text.replace('/n','')
    d['player_IPL_highestscore'] = soup.find_all("td", {"class":"text-right"})[43].text.replace('/n','')
    d['player_IPL_batavg'] = soup.find_all("td", {"class":"text-right"})[44].text.replace('/n','')
    d['player_IPL_ballsfaced'] = soup.find_all("td", {"class":"text-right"})[45].text.replace('/n','')
    d['player_IPL_strikerate'] = soup.find_all("td", {"class":"text-right"})[46].text.replace('/n','')
    d['player_IPL_100'] = soup.find_all("td", {"class":"text-right"})[47].text.replace('/n','')
    d['player_IPL_200'] = soup.find_all("td", {"class":"text-right"})[48].text.replace('/n','')
    d['player_IPL_50'] = soup.find_all("td", {"class":"text-right"})[49].text.replace('/n','')
    d['player_IPL_4s'] = soup.find_all("td", {"class":"text-right"})[50].text.replace('/n','')
    d['player_IPL_6s'] = soup.find_all("td", {"class":"text-right"})[51].text.replace('/n','')

    ##Bowling Summary

    #Test
    d['player_test_matches'] = soup.find_all("td", {"class":"text-right"})[52].text.replace('/n','')
    d['player_test_innings'] = soup.find_all("td", {"class":"text-right"})[53].text.replace('/n','')
    d['player_test_balls'] = soup.find_all("td", {"class":"text-right"})[54].text.replace('/n','')
    d['player_test_runs'] = soup.find_all("td", {"class":"text-right"})[55].text.replace('/n','')
    d['player_test_wkts'] = soup.find_all("td", {"class":"text-right"})[56].text.replace('/n','')
    d['player_test_bestbowlinginn'] = soup.find_all("td", {"class":"text-right"})[57].text.replace('/n','')
    d['player_test_bestbowlingmatch'] = soup.find_all("td", {"class":"text-right"})[58].text.replace('/n','')
    d['player_test_economy'] = soup.find_all("td", {"class":"text-right"})[59].text.replace('/n','')
    d['player_test_avg'] = soup.find_all("td", {"class":"text-right"})[60].text.replace('/n','')
    d['player_test_stkrate'] = soup.find_all("td", {"class":"text-right"})[61].text.replace('/n','')
    d['player_test_5wkts'] = soup.find_all("td", {"class":"text-right"})[62].text.replace('/n','')
    d['player_test_10wkts'] = soup.find_all("td", {"class":"text-right"})[63].text.replace('/n','')

    #ODI
    d['player_ODI_matches'] = soup.find_all("td", {"class":"text-right"})[64].text.replace('/n','')
    d['player_ODI_innings'] = soup.find_all("td", {"class":"text-right"})[65].text.replace('/n','')
    d['player_ODI_balls'] = soup.find_all("td", {"class":"text-right"})[66].text.replace('/n','')
    d['player_ODI_runs'] = soup.find_all("td", {"class":"text-right"})[67].text.replace('/n','')
    d['player_ODI_wkts'] = soup.find_all("td", {"class":"text-right"})[68].text.replace('/n','')
    d['player_ODI_bestbowlinginn'] = soup.find_all("td", {"class":"text-right"})[69].text.replace('/n','')
    d['player_ODI_bestbowlingmatch'] = soup.find_all("td", {"class":"text-right"})[70].text.replace('/n','')
    d['player_ODI_economy'] = soup.find_all("td", {"class":"text-right"})[71].text.replace('/n','')
    d['player_ODI_avg'] = soup.find_all("td", {"class":"text-right"})[72].text.replace('/n','')
    d['player_ODI_stkrate'] = soup.find_all("td", {"class":"text-right"})[73].text.replace('/n','')
    d['player_ODI_5wkts'] = soup.find_all("td", {"class":"text-right"})[74].text.replace('/n','')
    d['player_ODI_10wkts'] = soup.find_all("td", {"class":"text-right"})[75].text.replace('/n','')

    #T20
    d['player_T20_matches'] = soup.find_all("td", {"class":"text-right"})[76].text.replace('/n','')
    d['player_T20_innings'] = soup.find_all("td", {"class":"text-right"})[77].text.replace('/n','')
    d['player_T20_balls'] = soup.find_all("td", {"class":"text-right"})[78].text.replace('/n','')
    d['player_T20_runs'] = soup.find_all("td", {"class":"text-right"})[79].text.replace('/n','')
    d['player_T20_wkts'] = soup.find_all("td", {"class":"text-right"})[80].text.replace('/n','')
    d['player_T20_bestbowlinginn'] = soup.find_all("td", {"class":"text-right"})[81].text.replace('/n','')
    d['player_T20_bestbowlingmatch'] = soup.find_all("td", {"class":"text-right"})[82].text.replace('/n','')
    d['player_T20_economy'] = soup.find_all("td", {"class":"text-right"})[83].text.replace('/n','')
    d['player_T20_avg'] = soup.find_all("td", {"class":"text-right"})[84].text.replace('/n','')
    d['player_T20_stkrate'] = soup.find_all("td", {"class":"text-right"})[85].text.replace('/n','')
    d['player_T20_5wkts'] = soup.find_all("td", {"class":"text-right"})[86].text.replace('/n','')
    d['player_T20_10wkts'] = soup.find_all("td", {"class":"text-right"})[87].text.replace('/n','')

    #IPL
    d['player_IPL_matches'] = soup.find_all("td", {"class":"text-right"})[88].text.replace('/n','')
    d['player_IPL_innings'] = soup.find_all("td", {"class":"text-right"})[89].text.replace('/n','')
    d['player_IPL_balls'] = soup.find_all("td", {"class":"text-right"})[90].text.replace('/n','')
    d['player_IPL_runs'] = soup.find_all("td", {"class":"text-right"})[91].text.replace('/n','')
    d['player_IPL_wkts'] = soup.find_all("td", {"class":"text-right"})[92].text.replace('/n','')
    d['player_IPL_bestbowlinginn'] = soup.find_all("td", {"class":"text-right"})[93].text.replace('/n','')
    d['player_IPL_bestbowlingmatch'] = soup.find_all("td", {"class":"text-right"})[94].text.replace('/n','')
    d['player_IPL_economy'] = soup.find_all("td", {"class":"text-right"})[95].text.replace('/n','')
    d['player_IPL_avg'] = soup.find_all("td", {"class":"text-right"})[96].text.replace('/n','')
    d['player_IPL_stkrate'] = soup.find_all("td", {"class":"text-right"})[97].text.replace('/n','')
    d['player_IPL_5wkts'] = soup.find_all("td", {"class":"text-right"})[98].text.replace('/n','')
    d['player_IPL_10wkts'] = soup.find_all("td", {"class":"text-right"})[99].text.replace('/n','')
    #Append dictionary to list
    player_list.append(d)


#Create a pandas DataFrame to store data and save it to .csv
df = pd.DataFrame(player_list)
df.to_csv('Players_info.csv', index = False)
print("Success \n")

https://www.cricbuzz.com/profiles/1413/virat-kohli
https://www.cricbuzz.com/profiles/576/rohit-sharma
https://www.cricbuzz.com/profiles/9647/hardik-pandya
https://www.cricbuzz.com/profiles/587/ravindra-jadeja
https://www.cricbuzz.com/profiles/265/ms-dhoni
https://www.cricbuzz.com/profiles/1726/bhuvneshwar-kumar
https://www.cricbuzz.com/profiles/7909/mohammed-shami
https://www.cricbuzz.com/profiles/9311/jasprit-bumrah
https://www.cricbuzz.com/profiles/8292/kuldeep-yadav
https://www.cricbuzz.com/profiles/360/mohammad-hafeez
https://www.cricbuzz.com/profiles/33/shoaib-malik
https://www.cricbuzz.com/profiles/1629/angelo-mathews
https://www.cricbuzz.com/profiles/2281/thisara-perera
https://www.cricbuzz.com/profiles/111/lasith-malinga
https://www.cricbuzz.com/profiles/6692/moeen-ali
https://www.cricbuzz.com/profiles/6557/ben-stokes
https://www.cricbuzz.com/profiles/6670/chris-woakes
https://www.cricbuzz.com/profiles/9522/tom-curran
https://www.cricbuzz.com/profiles/479/liam-plunkett
https://